In [1]:
import sys
sys.path.insert(0, "S:\\YandexDisk\\code\\tasks")
from my_tools import *

In [3]:
%%writefile?

In [2]:
from my_tools import EDA

NameError: name 'true' is not defined

In [1]:
import pandas as pd
import numpy as np

In [2]:
%run "S:\YandexDisk\code\tasks\EDA.ipynb"

In [3]:
%run "S:\YandexDisk\code\tasks\Preparation.ipynb"

In [4]:
%run "S:\YandexDisk\code\tasks\Cleaning.ipynb"

In [5]:
%run "S:\YandexDisk\code\tasks\Models.ipynb"

In [6]:
%run "S:\YandexDisk\code\tasks\Selection.ipynb"

In [23]:
# user_ids = pd.read_csv('train.csv', chunksize=100000, usecols=['user_id'], low_memory=False)

# user_counts = None
# for chunk in user_ids:
#     if user_counts is None:
#         user_counts = chunk.user_id.value_counts()
#     else:
#         user_counts = user_counts.add(chunk.user_id.value_counts(), fill_value=0)

# user_counts = user_counts.to_dict()

In [24]:
dtypes = {
#     'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
#     'task_container_id': 'int16',
    'content_type_id': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float64',
    'prior_question_had_explanation': 'float32',
}

In [25]:
raw_train = pd.read_csv('train.csv',
                        nrows=10000,                        
                        dtype=dtypes,
                        usecols=dtypes.keys(),
                       low_memory=False)

In [26]:
raw_lectures = pd.read_csv('lectures.csv')
reduce_mem_usage(raw_lectures)

Mem. usage decreased to  0.01 Mb (58.8% reduction)


In [27]:
raw_questions = pd.read_csv('questions.csv')
reduce_mem_usage(raw_questions)

Mem. usage decreased to  0.18 Mb (65.0% reduction)


In [28]:
raw_lectures['type_of'] = raw_lectures['type_of'].replace(
    'solving question', 'solving_question')

In [29]:
questions = pd.get_dummies(raw_questions.fillna('-1'), columns=['part'],
                           dtype='int8')
lectures = pd.get_dummies(raw_lectures, columns=['tag', 'part', 'type_of',],
                          dtype='int8')

for k in range(188):
    questions['tag_'+str(k)] = np.int8(0)
    
for i in questions.index:
    for tag in questions.loc[i, 'tags'].split():
        if tag != '-1':
            questions.loc[i, 'tag_'+tag] = 1
questions.drop(columns='tags', inplace=True)

In [30]:
part_columns = [column for column in lectures.columns if column.startswith('part_')]
tag_l_columns = [column for column in lectures.columns if column.startswith('tag_')]
tag_q_columns = [column for column in questions.columns if column.startswith('tag_')]
types_of_columns = [column for column in lectures.columns if column.startswith('type_of_')]

In [31]:
reduce_mem_usage(questions)
reduce_mem_usage(lectures)

Mem. usage decreased to  2.58 Mb (0.0% reduction)


Mem. usage decreased to  0.07 Mb (0.0% reduction)


In [25]:
train_l = raw_train.loc[raw_train.content_type_id == 1].merge(
    lectures, how='left', left_on='content_id',
    right_on='lecture_id').drop(columns=['lecture_id'])

train = pd.concat([raw_train.loc[raw_train.content_type_id == 0], train_l
                   ]).sort_values(['timestamp', 'content_id'],
                                  ascending=True).reset_index(drop=True).drop(columns=['timestamp'])

train[types_of_columns] = train[types_of_columns].fillna(0).astype('int16')
train[tag_l_columns] = train[tag_l_columns].fillna(0).astype('int8')
train[part_columns] = train[part_columns].fillna(0).astype('int8')
del train_l

In [26]:
for col in progress_bar(part_columns):
    train[[
        f'{col}_sum_concept', f'{col}_sum_intention',
        f'{col}_sum_solving_question', f'{col}_sum_starter'
    ]] = train.loc[train[col] == 1, types_of_columns + ['user_id']].groupby(
        'user_id', sort=False).cumsum()

train.drop(columns=part_columns, inplace=True)

In [27]:
for col in progress_bar(tag_l_columns):
    if (train[col] == 0).all():
        train[[
            f'{col}_sum_concept', f'{col}_sum_intention',
            f'{col}_sum_solving_question', f'{col}_sum_starter'
        ]] = 0
    else:
        train[[
            f'{col}_sum_concept', f'{col}_sum_intention',
            f'{col}_sum_solving_question', f'{col}_sum_starter'
        ]] = train.loc[train[col] == 1,
                       ['user_id'] + types_of_columns].groupby(
                           'user_id', sort=False).cumsum()

train.drop(columns=tag_l_columns + types_of_columns, inplace=True)

In [28]:
exp_columns = [column for column in train.columns if (column.startswith('part_') or column.startswith('tag_'))]

In [29]:
train[exp_columns] = train[['user_id']+exp_columns].groupby('user_id', sort=False).ffill()
train[exp_columns] = train[exp_columns].fillna(0).astype('int16')

In [30]:
train = train.loc[train.content_type_id == 0].merge(
    questions, how='left', left_on='content_id',
    right_on='question_id').reset_index(drop=True).drop(
            columns=['question_id', 'content_type_id', 'correct_answer'])

In [31]:
train[part_columns+tag_q_columns] = train[part_columns+tag_q_columns].fillna(0).astype('int8')

In [32]:
train[['current_question_elapsed_time',
       'current_question_had_explanation']] = train[['user_id', 'bundle_id', 'prior_question_elapsed_time',
           'prior_question_had_explanation'
       ]].groupby(['user_id', 'bundle_id'], sort=False)[['user_id', 'prior_question_elapsed_time',
           'prior_question_had_explanation']].tail(1).groupby('user_id', sort=False).shift(-1)

In [33]:
train[['current_question_elapsed_time',
       'current_question_had_explanation']] = train[[
           'user_id', 'current_question_elapsed_time',
           'current_question_had_explanation'
       ]].groupby('user_id', sort=False).bfill()

In [34]:
q_stats = train[[
    'content_id', 'answered_correctly', 'current_question_elapsed_time',
    'current_question_had_explanation'
]].groupby('content_id').mean()

q_stats.columns = ['q_mean_answer', 'q_mean_time', 'q_mean_explain']

In [35]:
for col in progress_bar(part_columns):
    train[[
        #         f'{col}_mean_answer',
        f'{col}_mean_time',
        f'{col}_mean_explain'
    ]] = train.loc[train[col] == 1, [
        'user_id',
        #         'answered_correctly',
        'current_question_elapsed_time',
        'current_question_had_explanation'
    ]].groupby('user_id', sort=False)[[
        #         'answered_correctly',
        'current_question_elapsed_time',
        'current_question_had_explanation'
    ]].expanding().mean().groupby('user_id', sort=False).shift().reset_index(
        level=0, drop=True)

In [36]:
for col in progress_bar(tag_q_columns):
    if (train[col] == 0).all():
        train[[
            #         f'{col}_mean_answer',
            f'{col}_mean_time',
            f'{col}_mean_explain'
        ]] = 0
    else:
        train[[
            #         f'{col}_mean_answer',
            f'{col}_mean_time',
            f'{col}_mean_explain'
        ]] = train.loc[train[col] == 1, [
            'user_id',
            #         'answered_correctly',
            'current_question_elapsed_time',
            'current_question_had_explanation'
        ]].groupby('user_id', sort=False)[[
            #         'answered_correctly',
            'current_question_elapsed_time',
            'current_question_had_explanation'
        ]].expanding().mean().groupby('user_id',
                                      sort=False).shift().reset_index(
                                          level=0, drop=True)

train.drop(columns=[
    'current_question_elapsed_time', 'current_question_had_explanation'
],
           inplace=True)

In [37]:
exp_columns = [column for column in train.columns if (column.startswith('part_') or column.startswith('tag_'))]
exp_columns = list(set(exp_columns) - set(part_columns) - set(tag_q_columns))

In [38]:
train[exp_columns] = train[['user_id']+exp_columns].groupby('user_id', sort=False).ffill()
train[exp_columns] = train[exp_columns].fillna(0)

In [39]:
train = train.merge(q_stats, how='left', left_on='content_id', right_index=True)

In [40]:
train.fillna(0, inplace=True)

In [41]:
reduce_mem_usage(train)

Mem. usage decreased to 18.90 Mb (56.2% reduction)


In [44]:
X = train.drop(columns=[
    'user_id',
    'row_id',
#     'content_id',
#         'bundle_id',
    'task_container_id',
    'answered_correctly'])
y = train.answered_correctly

In [51]:
X_minmax = scale_it(X, X.columns, 'minmax')
reduce_mem_usage(X_minmax)

Mem. usage decreased to 23.22 Mb (68.3% reduction)


In [58]:
model = LogisticRegression(C=0.7140720176503248, intercept_scaling=6.118362684156746,
                    l1_ratio=0.630368291400373, n_jobs=-1, penalty='elasticnet',
                    random_state=13, solver='saga')
model.fit(X, y)

C:\Users\done\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



LogisticRegression(C=0.7140720176503248, intercept_scaling=6.118362684156746,
                   l1_ratio=0.630368291400373, n_jobs=-1, penalty='elasticnet',
                   random_state=13, solver='saga')

In [446]:
X_robust = scale_it(X, X.columns, 'robust')
reduce_mem_usage(X_robust)

Mem. usage decreased to 331.53 Mb (63.9% reduction)


In [447]:
X_standard = scale_it(X, X.columns, 'standard')
reduce_mem_usage(X_standard)

Mem. usage decreased to 695.01 Mb (24.4% reduction)


In [409]:
mm = Lin_Clf({'log': LogisticRegression(C=0.7140720176503248, intercept_scaling=6.118362684156746,
                    l1_ratio=0.630368291400373, n_jobs=-1, penalty='l1',
                    random_state=13, solver='saga')})
mm.val(X_minmax, y)

log validated in 650.700119972229


log    0.8082
Name: val_scores, dtype: float64

In [439]:
%run "S:\YandexDisk\code\tasks\Selection.ipynb"

In [451]:
bo = BO_Clf(X_minmax, y)

In [452]:
bo.fit('log', 200)

[INFO 11-26 00:20:03] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[WARNING 11-26 00:20:03] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 11-26 00:20:03] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.


Optimizing log


[INFO 11-26 00:20:03] ax.service.ax_client: Generated new trial 0 with parameters {'C': 6.14, 'intercept_scaling': 2.41, 'l1_ratio': 0.68, 'penalty': 'elasticnet', 'dual': True, 'solver': 'newton-cg'}.
[INFO 11-26 00:20:12] ax.service.ax_client: Completed trial 0 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 00:20:12] ax.service.ax_client: Generated new trial 1 with parameters {'C': 4.3, 'intercept_scaling': 4.32, 'l1_ratio': 0.06, 'penalty': 'elasticnet', 'dual': True, 'solver': 'sag'}.
[INFO 11-26 00:20:20] ax.service.ax_client: Completed trial 1 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 00:20:20] ax.service.ax_client: Generated new trial 2 with parameters {'C': 8.06, 'intercept_scaling': 7.81, 'l1_ratio': 0.92, 'penalty': 'l2', 'dual': False, 'solver': 'saga'}.
[INFO 11-26 00:23:16] ax.service.ax_client: Completed trial 2 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 00:23:16] ax.service.ax_client: Generated new trial 3 with parameters {'C': 8.91, 'intercept_scaling': 6.3

[INFO 11-26 01:49:34] ax.service.ax_client: Generated new trial 28 with parameters {'C': 2.04, 'intercept_scaling': 7.04, 'l1_ratio': 0.35, 'penalty': 'elasticnet', 'dual': True, 'solver': 'lbfgs'}.
[INFO 11-26 01:49:43] ax.service.ax_client: Completed trial 28 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 01:49:43] ax.service.ax_client: Generated new trial 29 with parameters {'C': 7.78, 'intercept_scaling': 9.06, 'l1_ratio': 0.91, 'penalty': 'l1', 'dual': True, 'solver': 'saga'}.
[INFO 11-26 01:49:53] ax.service.ax_client: Completed trial 29 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 01:49:53] ax.service.ax_client: Generated new trial 30 with parameters {'C': 4.04, 'intercept_scaling': 3.08, 'l1_ratio': 0.11, 'penalty': 'none', 'dual': False, 'solver': 'lbfgs'}.
[INFO 11-26 01:50:26] ax.service.ax_client: Completed trial 30 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 01:50:26] ax.service.ax_client: Generated new trial 31 with parameters {'C': 6.92, 'intercept_scaling': 7.5

[INFO 11-26 03:34:58] ax.service.ax_client: Generated new trial 56 with parameters {'C': 3.79, 'intercept_scaling': 8.34, 'l1_ratio': 0.32, 'penalty': 'elasticnet', 'dual': True, 'solver': 'newton-cg'}.
[INFO 11-26 03:35:08] ax.service.ax_client: Completed trial 56 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 03:35:08] ax.service.ax_client: Generated new trial 57 with parameters {'C': 5.71, 'intercept_scaling': 5.26, 'l1_ratio': 0.92, 'penalty': 'elasticnet', 'dual': True, 'solver': 'lbfgs'}.
[INFO 11-26 03:35:16] ax.service.ax_client: Completed trial 57 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 03:35:16] ax.service.ax_client: Generated new trial 58 with parameters {'C': 1.97, 'intercept_scaling': 1.88, 'l1_ratio': 0.09, 'penalty': 'l2', 'dual': False, 'solver': 'saga'}.
[INFO 11-26 03:38:10] ax.service.ax_client: Completed trial 58 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 03:38:11] ax.service.ax_client: Generated new trial 59 with parameters {'C': 0.94, 'intercept_sca

[INFO 11-26 03:52:45] ax.service.ax_client: Generated new trial 84 with parameters {'C': 8.81, 'intercept_scaling': 8.1, 'l1_ratio': 0.12, 'penalty': 'elasticnet', 'dual': False, 'solver': 'saga'}.
[INFO 11-26 03:57:47] ax.service.ax_client: Completed trial 84 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 03:57:47] ax.service.ax_client: Generated new trial 85 with parameters {'C': 0.71, 'intercept_scaling': 6.12, 'l1_ratio': 0.63, 'penalty': 'l1', 'dual': False, 'solver': 'saga'}.
[INFO 11-26 04:02:30] ax.service.ax_client: Completed trial 85 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 04:02:31] ax.service.ax_client: Generated new trial 86 with parameters {'C': 6.96, 'intercept_scaling': 2.11, 'l1_ratio': 0.35, 'penalty': 'none', 'dual': True, 'solver': 'lbfgs'}.
[INFO 11-26 04:02:41] ax.service.ax_client: Completed trial 86 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 04:02:41] ax.service.ax_client: Generated new trial 87 with parameters {'C': 6.8, 'intercept_scaling': 2.87

[INFO 11-26 05:22:57] ax.service.ax_client: Generated new trial 112 with parameters {'C': 7.05, 'intercept_scaling': 6.8, 'l1_ratio': 0.08, 'penalty': 'elasticnet', 'dual': False, 'solver': 'sag'}.
[INFO 11-26 05:23:05] ax.service.ax_client: Completed trial 112 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 05:23:06] ax.service.ax_client: Generated new trial 113 with parameters {'C': 2.79, 'intercept_scaling': 9.92, 'l1_ratio': 0.66, 'penalty': 'l1', 'dual': False, 'solver': 'newton-cg'}.
[INFO 11-26 05:23:14] ax.service.ax_client: Completed trial 113 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 05:23:14] ax.service.ax_client: Generated new trial 114 with parameters {'C': 9.03, 'intercept_scaling': 3.31, 'l1_ratio': 0.32, 'penalty': 'l2', 'dual': True, 'solver': 'saga'}.
[INFO 11-26 05:23:22] ax.service.ax_client: Completed trial 114 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 05:23:23] ax.service.ax_client: Generated new trial 115 with parameters {'C': 8.02, 'intercept_scaling

[INFO 11-26 06:27:18] ax.service.ax_client: Generated new trial 140 with parameters {'C': 0.13, 'intercept_scaling': 4.09, 'l1_ratio': 0.59, 'penalty': 'elasticnet', 'dual': True, 'solver': 'newton-cg'}.
[INFO 11-26 06:27:26] ax.service.ax_client: Completed trial 140 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 06:27:27] ax.service.ax_client: Generated new trial 141 with parameters {'C': 9.39, 'intercept_scaling': 2.34, 'l1_ratio': 0.16, 'penalty': 'none', 'dual': True, 'solver': 'sag'}.
[INFO 11-26 06:27:35] ax.service.ax_client: Completed trial 141 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 06:27:35] ax.service.ax_client: Generated new trial 142 with parameters {'C': 3.13, 'intercept_scaling': 5.74, 'l1_ratio': 0.83, 'penalty': 'l2', 'dual': False, 'solver': 'saga'}.
[INFO 11-26 06:30:30] ax.service.ax_client: Completed trial 142 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 06:30:31] ax.service.ax_client: Generated new trial 143 with parameters {'C': 0.4, 'intercept_scali

[INFO 11-26 07:04:15] ax.service.ax_client: Generated new trial 168 with parameters {'C': 3.37, 'intercept_scaling': 0.36, 'l1_ratio': 0.62, 'penalty': 'elasticnet', 'dual': True, 'solver': 'saga'}.
[INFO 11-26 07:04:23] ax.service.ax_client: Completed trial 168 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 07:04:24] ax.service.ax_client: Generated new trial 169 with parameters {'C': 6.46, 'intercept_scaling': 3.56, 'l1_ratio': 0.14, 'penalty': 'elasticnet', 'dual': True, 'solver': 'sag'}.
[INFO 11-26 07:04:32] ax.service.ax_client: Completed trial 169 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 07:04:33] ax.service.ax_client: Generated new trial 170 with parameters {'C': 0.2, 'intercept_scaling': 9.52, 'l1_ratio': 0.85, 'penalty': 'none', 'dual': False, 'solver': 'newton-cg'}.
[INFO 11-26 07:46:19] ax.service.ax_client: Completed trial 170 with data: {'ROC_AUC': (0.81, 0.0)}.
[INFO 11-26 07:46:20] ax.service.ax_client: Generated new trial 171 with parameters {'C': 1.85, 'interce

[INFO 11-26 08:07:46] ax.service.ax_client: Generated new trial 196 with parameters {'C': 8.36, 'intercept_scaling': 0.75, 'l1_ratio': 0.81, 'penalty': 'elasticnet', 'dual': False, 'solver': 'lbfgs'}.
[INFO 11-26 08:07:53] ax.service.ax_client: Completed trial 196 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 08:07:54] ax.service.ax_client: Generated new trial 197 with parameters {'C': 1.45, 'intercept_scaling': 2.54, 'l1_ratio': 0.43, 'penalty': 'l1', 'dual': False, 'solver': 'newton-cg'}.
[INFO 11-26 08:08:02] ax.service.ax_client: Completed trial 197 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 08:08:03] ax.service.ax_client: Generated new trial 198 with parameters {'C': 5.21, 'intercept_scaling': 9.13, 'l1_ratio': 0.58, 'penalty': 'l2', 'dual': True, 'solver': 'saga'}.
[INFO 11-26 08:08:11] ax.service.ax_client: Completed trial 198 with data: {'ROC_AUC': (nan, nan)}.
[INFO 11-26 08:08:12] ax.service.ax_client: Generated new trial 199 with parameters {'C': 6.04, 'intercept_scal

In [453]:
bo.best_estimators_

{'log': LogisticRegression(C=0.7140720176503248, intercept_scaling=6.118362684156746,
                    l1_ratio=0.630368291400373, n_jobs=-1, penalty='l1',
                    random_state=13, solver='saga')}

In [454]:
bo.fit('sgd', 200)

[INFO 11-26 08:08:23] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[WARNING 11-26 08:08:23] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 11-26 08:08:23] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.


Optimizing sgd


[INFO 11-26 08:08:23] ax.service.ax_client: Generated new trial 0 with parameters {'alpha': 0.02, 'l1_ratio': 0.65, 'tol': 0.03, 'eta0': 4.62, 'loss': 'hinge', 'penalty': 'l2', 'learning_rate': 'adaptive'}.
[INFO 11-26 08:09:35] ax.service.ax_client: Completed trial 0 with data: {'ROC_AUC': (0.8, 0.0)}.
[INFO 11-26 08:09:35] ax.service.ax_client: Generated new trial 1 with parameters {'alpha': 0.06, 'l1_ratio': 0.92, 'tol': 0.09, 'eta0': 0.77, 'loss': 'log', 'penalty': 'l1', 'learning_rate': 'optimal'}.
[INFO 11-26 08:09:56] ax.service.ax_client: Completed trial 1 with data: {'ROC_AUC': (0.5, 0.0)}.
[INFO 11-26 08:09:56] ax.service.ax_client: Generated new trial 2 with parameters {'alpha': 0.03, 'l1_ratio': 0.0, 'tol': 0.02, 'eta0': 7.66, 'loss': 'modified_huber', 'penalty': 'l1', 'learning_rate': 'adaptive'}.
[INFO 11-26 08:11:55] ax.service.ax_client: Completed trial 2 with data: {'ROC_AUC': (0.8, 0.0)}.
[INFO 11-26 08:11:55] ax.service.ax_client: Generated new trial 3 with parameter

[INFO 11-26 08:30:54] ax.service.ax_client: Completed trial 26 with data: {'ROC_AUC': (0.65, 0.15)}.
[INFO 11-26 08:30:55] ax.service.ax_client: Generated new trial 27 with parameters {'alpha': 0.03, 'l1_ratio': 0.69, 'tol': 0.05, 'eta0': 7.43, 'loss': 'modified_huber', 'penalty': 'l2', 'learning_rate': 'optimal'}.
[INFO 11-26 08:31:12] ax.service.ax_client: Completed trial 27 with data: {'ROC_AUC': (0.8, 0.0)}.
[INFO 11-26 08:31:12] ax.service.ax_client: Generated new trial 28 with parameters {'alpha': 0.05, 'l1_ratio': 0.36, 'tol': 0.05, 'eta0': 4.13, 'loss': 'squared_hinge', 'penalty': 'l1', 'learning_rate': 'invscaling'}.
[INFO 11-26 08:31:32] ax.service.ax_client: Completed trial 28 with data: {'ROC_AUC': (0.73, 0.01)}.
[INFO 11-26 08:31:32] ax.service.ax_client: Generated new trial 29 with parameters {'alpha': 0.02, 'l1_ratio': 0.09, 'tol': 0.01, 'eta0': 0.48, 'loss': 'log', 'penalty': 'elasticnet', 'learning_rate': 'optimal'}.
[INFO 11-26 08:31:54] ax.service.ax_client: Complete

[INFO 11-26 08:46:44] ax.service.ax_client: Generated new trial 53 with parameters {'alpha': 0.09, 'l1_ratio': 0.02, 'tol': 0.09, 'eta0': 5.33, 'loss': 'hinge', 'penalty': 'l2', 'learning_rate': 'optimal'}.
[INFO 11-26 08:47:01] ax.service.ax_client: Completed trial 53 with data: {'ROC_AUC': (0.79, 0.0)}.
[INFO 11-26 08:47:01] ax.service.ax_client: Generated new trial 54 with parameters {'alpha': 0.0, 'l1_ratio': 0.94, 'tol': 0.01, 'eta0': 3.74, 'loss': 'perceptron', 'penalty': 'elasticnet', 'learning_rate': 'invscaling'}.
[INFO 11-26 08:47:20] ax.service.ax_client: Completed trial 54 with data: {'ROC_AUC': (0.78, 0.01)}.
[INFO 11-26 08:47:20] ax.service.ax_client: Generated new trial 55 with parameters {'alpha': 0.07, 'l1_ratio': 0.25, 'tol': 0.0, 'eta0': 9.84, 'loss': 'modified_huber', 'penalty': 'l2', 'learning_rate': 'constant'}.
[INFO 11-26 08:47:36] ax.service.ax_client: Completed trial 55 with data: {'ROC_AUC': (0.57, 0.01)}.
[INFO 11-26 08:47:37] ax.service.ax_client: Generated

[INFO 11-26 09:02:42] ax.service.ax_client: Generated new trial 79 with parameters {'alpha': 0.04, 'l1_ratio': 0.44, 'tol': 0.02, 'eta0': 0.46, 'loss': 'log', 'penalty': 'elasticnet', 'learning_rate': 'adaptive'}.
[INFO 11-26 09:04:35] ax.service.ax_client: Completed trial 79 with data: {'ROC_AUC': (0.8, 0.0)}.
[INFO 11-26 09:04:36] ax.service.ax_client: Generated new trial 80 with parameters {'alpha': 0.07, 'l1_ratio': 0.61, 'tol': 0.08, 'eta0': 8.63, 'loss': 'perceptron', 'penalty': 'l1', 'learning_rate': 'invscaling'}.
[INFO 11-26 09:04:56] ax.service.ax_client: Completed trial 80 with data: {'ROC_AUC': (0.5, 0.0)}.
[INFO 11-26 09:04:57] ax.service.ax_client: Generated new trial 81 with parameters {'alpha': 0.0, 'l1_ratio': 0.84, 'tol': 0.03, 'eta0': 7.26, 'loss': 'squared_hinge', 'penalty': 'elasticnet', 'learning_rate': 'optimal'}.
[INFO 11-26 09:05:17] ax.service.ax_client: Completed trial 81 with data: {'ROC_AUC': (0.69, 0.0)}.
[INFO 11-26 09:05:17] ax.service.ax_client: Generat

[INFO 11-26 09:23:05] ax.service.ax_client: Completed trial 105 with data: {'ROC_AUC': (0.57, 0.01)}.
[INFO 11-26 09:23:05] ax.service.ax_client: Generated new trial 106 with parameters {'alpha': 0.05, 'l1_ratio': 0.31, 'tol': 0.0, 'eta0': 2.06, 'loss': 'perceptron', 'penalty': 'l1', 'learning_rate': 'optimal'}.
[INFO 11-26 09:23:26] ax.service.ax_client: Completed trial 106 with data: {'ROC_AUC': (0.5, 0.0)}.
[INFO 11-26 09:23:27] ax.service.ax_client: Generated new trial 107 with parameters {'alpha': 0.0, 'l1_ratio': 0.51, 'tol': 0.08, 'eta0': 4.56, 'loss': 'log', 'penalty': 'elasticnet', 'learning_rate': 'optimal'}.
[INFO 11-26 09:23:49] ax.service.ax_client: Completed trial 107 with data: {'ROC_AUC': (0.8, 0.0)}.
[INFO 11-26 09:23:49] ax.service.ax_client: Generated new trial 108 with parameters {'alpha': 0.08, 'l1_ratio': 0.42, 'tol': 0.02, 'eta0': 6.41, 'loss': 'hinge', 'penalty': 'l2', 'learning_rate': 'invscaling'}.
[INFO 11-26 09:24:06] ax.service.ax_client: Completed trial 10

[INFO 11-26 09:36:09] ax.service.ax_client: Completed trial 131 with data: {'ROC_AUC': (0.79, 0.0)}.
[INFO 11-26 09:36:09] ax.service.ax_client: Generated new trial 132 with parameters {'alpha': 0.09, 'l1_ratio': 0.82, 'tol': 0.09, 'eta0': 4.77, 'loss': 'hinge', 'penalty': 'l1', 'learning_rate': 'adaptive'}.
[INFO 11-26 09:37:59] ax.service.ax_client: Completed trial 132 with data: {'ROC_AUC': (0.5, 0.0)}.
[INFO 11-26 09:37:59] ax.service.ax_client: Generated new trial 133 with parameters {'alpha': 0.03, 'l1_ratio': 0.62, 'tol': 0.04, 'eta0': 1.13, 'loss': 'hinge', 'penalty': 'l2', 'learning_rate': 'constant'}.
[INFO 11-26 09:38:15] ax.service.ax_client: Completed trial 133 with data: {'ROC_AUC': (0.57, 0.01)}.
[INFO 11-26 09:38:16] ax.service.ax_client: Generated new trial 134 with parameters {'alpha': 0.05, 'l1_ratio': 0.46, 'tol': 0.05, 'eta0': 7.96, 'loss': 'perceptron', 'penalty': 'l1', 'learning_rate': 'optimal'}.
[INFO 11-26 09:38:36] ax.service.ax_client: Completed trial 134 wi

[INFO 11-26 09:54:23] ax.service.ax_client: Completed trial 157 with data: {'ROC_AUC': (0.61, 0.0)}.
[INFO 11-26 09:54:24] ax.service.ax_client: Generated new trial 158 with parameters {'alpha': 0.01, 'l1_ratio': 0.04, 'tol': 0.0, 'eta0': 9.41, 'loss': 'modified_huber', 'penalty': 'l2', 'learning_rate': 'invscaling'}.
[INFO 11-26 09:54:40] ax.service.ax_client: Completed trial 158 with data: {'ROC_AUC': (0.69, 0.01)}.
[INFO 11-26 09:54:41] ax.service.ax_client: Generated new trial 159 with parameters {'alpha': 0.08, 'l1_ratio': 0.71, 'tol': 0.04, 'eta0': 9.19, 'loss': 'log', 'penalty': 'elasticnet', 'learning_rate': 'constant'}.
[INFO 11-26 09:55:04] ax.service.ax_client: Completed trial 159 with data: {'ROC_AUC': (0.5, 0.0)}.
[INFO 11-26 09:55:05] ax.service.ax_client: Generated new trial 160 with parameters {'alpha': 0.01, 'l1_ratio': 0.36, 'tol': 0.06, 'eta0': 2.36, 'loss': 'hinge', 'penalty': 'elasticnet', 'learning_rate': 'invscaling'}.
[INFO 11-26 09:55:25] ax.service.ax_client: 

KeyboardInterrupt: 

In [ ]:
bo.best_estimators_

In [ ]:
bo.fit('lsvm', 200)

In [ ]:
bo.best_estimators_

In [ ]:
bo.fit('perceptron', 200)

In [ ]:
bo.best_estimators_

In [448]:
std = Lin_Clf()
std.val(X_standard, y)

log validated in 65.66251850128174


log    0.8068
Name: val_scores, dtype: float64

In [449]:
rob = Lin_Clf()
rob.val(X_robust, y)

log validated in 67.44165062904358
ridge validated in 29.524224519729614
dt validated in 60.79762053489685
et validated in 16.11095690727234
passagr validated in 25.93632197380066
lsvm validated in 34.09196972846985
svm validated in 83.44764304161072
perceptron validated in 22.711088180541992
sgd validated in 319.42267394065857


ridge         0.8070
lsvm          0.6295
dt            0.6271
passagr       0.6247
sgd           0.5822
et            0.5790
perceptron    0.5560
log           0.5539
svm           0.4843
Name: val_scores, dtype: float64

In [450]:
mm = Lin_Clf()
mm.val(X_minmax, y)

log validated in 63.709728956222534
ridge validated in 28.697229385375977
dt validated in 59.68072175979614
et validated in 17.46940541267395
passagr validated in 25.569925546646118
lsvm validated in 40.25847840309143
svm validated in 85.58344054222107
perceptron validated in 23.322450160980225
sgd validated in 61.86084222793579


log           0.8081
ridge         0.8070
sgd           0.8005
lsvm          0.7617
perceptron    0.7590
passagr       0.7485
svm           0.6462
dt            0.6267
et            0.5751
Name: val_scores, dtype: float64